# Tutorial: Entendimiento de los datos con PySpark

Continuando con el proyecto de consultoria de Wide World Importers, el primer paso para iniciar la comprensión de los datos es explorar y entender las fuentes de datos disponibles. Note que esto también nos ayuda a comprender mejor la organización.

## Configuración e importe de paquetes
Se utilizará el paquete de pandas profiling para apoyar el análisis estadístico, y se importan los paquetes de python
necesarios

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rcParams['figure.max_open_warning'] = 30  # or any other number 
matplotlib.use('TkAgg')  # Usando el backend TkAgg

Configuración del controlador e inicio de sesion Spark

In [3]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [4]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = 'Estudiante_67_202315'
db_psswd = 'aabb1122'

PATH='./'

### Conexión a fuente de datos y acceso a los datos

#### Conexión a fuente de datos
A continuación encuentra las funciones para conectarse a la fuente de datos (archivo csv o base de datos) y retornar un dataframe que es el que se utilizará posteriormente para manipular los datos.

In [6]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

#### Cargue de datos
Para consultar desde la base de datos se puede acceder directamente a la tabla dado su nombre o se puede especificar la sentencia SQL de un "SELECT" completo para traer la información total o parcial de la tabla que se requiere. A continuación un ejemplo de cada uno de estos usos. 
Acceso directo para el caso de órdenes y acceso con sentencia SQL para el caso de detallesOrdenes.

### Completitud y validez

In [7]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter(df[c].isNull()).count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
    return resultados 

# Checking "" values 
def contar_vacios_str(df, printing = True):
    resultados = []
    for c in df.columns:
        vacios = df.filter((df[c] == "")).count()
        if vacios != 0: 
            if printing: 
                print('Number of "" values for column ' + c + ': ' + str(vacios))
            resultados.append(vacios)
    return resultados 


def cardinalidad(df):
    resultado = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
    return resultado

# 5. Tarea
Espacio para desarrollar la tarea propuesta 

### Perfilamiento de datos

#### Entendimiento general de datos

In [8]:
# Getting Df from WWImportersTransactional.movimientosCopia 
sql_movimientosCopia = 'WWImportersTransactional.movimientosCopia'
movimientosCopia = obtener_dataframe_de_bd(db_connection_string, sql_movimientosCopia, db_user, db_psswd) 
# Exploring the table 
movimientosCopia.show(5) 

+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|TransaccionProductoID|ProductoID|TipoTransaccionID|ClienteID|InvoiceID|ProveedorID|OrdenDeCompraID|FechaTransaccion|Cantidad|
+---------------------+----------+-----------------+---------+---------+-----------+---------------+----------------+--------+
|               118903|       217|               10|    476.0|  24904.0|           |               |     Apr 25,2014|   -40.0|
|               286890|       135|               10|     33.0|  60117.0|           |               |     Dec 10,2015|    -7.0|
|               285233|       111|               10|    180.0|  59768.0|           |               |     Dec 04,2015|    -2.0|
|               290145|       213|               10|     33.0|  60795.0|           |               |     Dec 23,2015|    -3.0|
|               247492|        90|               10|     55.0|  51851.0|           |               |     Jul 27

In [9]:
# Getting each column's type 
print("Columns' type") 
print(movimientosCopia.schema) 
# Printing df's dimensions 
print("\n Df's dimensions") 
print((movimientosCopia.count(), len(movimientosCopia.columns))) 

Columns' type
StructType(List(StructField(TransaccionProductoID,IntegerType,true),StructField(ProductoID,IntegerType,true),StructField(TipoTransaccionID,IntegerType,true),StructField(ClienteID,DoubleType,true),StructField(InvoiceID,DoubleType,true),StructField(ProveedorID,StringType,true),StructField(OrdenDeCompraID,StringType,true),StructField(FechaTransaccion,StringType,true),StructField(Cantidad,DoubleType,true)))

 Df's dimensions
(204292, 9)


#### Revisión de reglas de negocio

In [10]:
"""
    1) The maximum amount of products moved is 50 million per transaction 
"""
print("\n1) The maximum amount of products moved is 50 million per transaction ".center(20, '*')) 
max_valor = movimientosCopia.agg({"Cantidad": "max"}).collect()[0][0]
# Printing the max value 
print("Max Value for 'Cantidad':", max_valor)

"""
    Given that the maximum value for 'cantidad' in a transaction is 
    67,368.0, it can be assumed that this rule hasn't been broken, and 
    as of now, there are no transactions that exceed this limit.
"""

"""
    2) There are 236,668 product movements carried out since 2013 
"""
print("\n2) There are 236,668 product movements carried out since 2013".center(20, '*')) 

print(f"\nAmount of registers: {movimientosCopia.count()}") 
print("Min Date:")
min_fecha = movimientosCopia.agg({"FechaTransaccion": "min"}).collect()[0][0]

# Print the min date 
print("Minimum date 'FechaTransaccion':", min_fecha)
"""
    It was said that there were 236,668 product movements, but based on 
    the actual dimensions, it appears to be only 204,292. Additionally, 
    it was mentioned that the minimum date was in 2013, and in a way, 
    this is accurate since the first transaction occurred on 2013-12-31 
    07:00:00.0000000. Therefore, this part of the rule can be considered 
    valid.
"""

"""
    3) The movements are related to 228,265 clients
""" 
print("\n3) The movements are related to 228,265 clients".center(20, '*')) 
# Getting the number of different ClienteID
client_count = movimientosCopia.select("ClienteID").count() 
print(f"\nNumber of distinct clients: {client_count}") 

"""
    There are 204,292 distinct clients, so this rule isn't true. It will 
    needed to ask about this rule to know if there are missing info 
    because of this difference and also the minimum transaction date 
    dilemma and maybe the first one since the max "Cantidad" value 
    isn't even near to the limit stablished by the rule. 
"""

"""
    4) The date format used is YYYY-MM-DD HH:MM:SS 
""" 
print("\n4) The date format used is YYYY-MM-DD HH:MM:SS".center(20, '*')) 

# Filtering those columns that satisfied the format 
regex = "[0-2]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])"
fulfillsFormat_FT = movimientosCopia.filter(movimientosCopia["FechaTransaccion"].rlike(regex)) 
print("\nTotal number of orders: " + str(movimientosCopia.count()) + ", number of orders with correct format: " + str(fulfillsFormat_FT.count()))

# Checking those rows that doesn't fit the format 
noFulfillsFormat_FT = movimientosCopia.filter(~movimientosCopia["FechaTransaccion"].rlike(regex)) 
noFulfillsFormat_FT.count(), noFulfillsFormat_FT.select(col("FechaTransaccion")).show() 
print(f"Number of rows that doesn't fit the correct format: {noFulfillsFormat_FT.count()}") 

# Checking if there are another different kind of values in this column 
print(f"Checking if there are some different values in the column: ")
movimientosCopia.where(length(col("FechaTransaccion")) <= 10).show()

"""
   There are identified 64,254 entries that deviate from the specified date 
   format. However, it appears that these discrepancies are solely due 
   to differences in formatting. These issues can be easily resolved, so 
   notifying the business is more about providing information rather 
   than seeking clarification. 
   
"""

"""
    5) We currently have 13 suppliers
""" 
print("\n5) We currently have 13 suppliers".center(20, '*'))  

# Checking if there are some errors on the column ProveedorID
filtered_df = movimientosCopia.filter(movimientosCopia["ProveedorID"] == movimientosCopia["ClienteID"])

# Perform the count operation
result_count = filtered_df.count() 

print(f"There are {result_count} rows that have ProveedorID == ClienteID") 

# Getting the number of different ProveedorID 
result_count = movimientosCopia.select("ProveedorID").distinct().count() 
print(f"There are {result_count} distinct ProveedorID") 

"""
    There are only 4 different suppliers so this rule isn't satisfied. Maybe 
    this reinforces the idea that there are missing values!  
"""


1) The maximum amount of products moved is 50 million per transaction 
Max Value for 'Cantidad': 67368.0

2) There are 236,668 product movements carried out since 2013

Amount of registers: 204292
Min Date:
Minimum date 'FechaTransaccion': 2013-12-31 07:00:00.0000000

3) The movements are related to 228,265 clients

Number of distinct clients: 204292

4) The date format used is YYYY-MM-DD HH:MM:SS

Total number of orders: 204292, number of orders with correct format: 140038
+----------------+
|FechaTransaccion|
+----------------+
|     Apr 25,2014|
|     Dec 10,2015|
|     Dec 04,2015|
|     Dec 23,2015|
|     Jul 27,2015|
|     Sep 15,2014|
|     Aug 04,2015|
|     Feb 23,2015|
|     May 01,2015|
|     Jan 08,2016|
|     Mar 26,2014|
|     Jul 31,2015|
|     Sep 02,2014|
|     Mar 15,2016|
|     May 28,2016|
|     Sep 09,2015|
|     May 23,2014|
|     Aug 20,2014|
|     Jan 21,2015|
|     Dec 29,2015|
+----------------+
only showing top 20 rows

Number of rows that doesn't fit the co

"\n    There are only 4 different suppliers so this rule isn't satisfied. Maybe \n    this reinforces the idea that there are missing values!  \n"

### Análisis descriptivo

In [11]:
movimientosCopia.summary().show() 

"""
    It seems that every column has data but some of 
    them has outliers, like the column "Cantidad" 
    which its minimum value is -360.0 and its maximum 
    is 67368.0. Of course, it will be checked later. 
"""

+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|summary|TransaccionProductoID|        ProductoID|  TipoTransaccionID|         ClienteID|        InvoiceID|      ProveedorID|   OrdenDeCompraID|    FechaTransaccion|         Cantidad|
+-------+---------------------+------------------+-------------------+------------------+-----------------+-----------------+------------------+--------------------+-----------------+
|  count|               204292|            204292|             204292|            204292|           204292|           204292|            204292|              204292|           204292|
|   mean|   212458.04047637695|110.70090850351458| 10.035253460732676| 517.3252941867523|42957.26929590978|4.951898734177215|1345.9973277074544|                null|719.4997650421946|
| stddev|    71352.37579752573| 63.49014746219581|0.18563716955046372|353.501369

'\n    It seems that every column has data but some of \n    them has outliers, like the column "Cantidad" \n    which its minimum value is -360.0 and its maximum \n    is 67368.0. Of course, it will be checked later. \n'

In [12]:
fig1, ax1 = plt.subplots()
ax1.set_title('Cantidad')
ax1.boxplot(np.array(movimientosCopia.select('Cantidad').collect()))

{'whiskers': [<matplotlib.lines.Line2D at 0x28f81dfbf48>,
 'caps': [<matplotlib.lines.Line2D at 0x28f81dfb7c8>,
 'boxes': [<matplotlib.lines.Line2D at 0x28ff6610948>],
 'medians': [<matplotlib.lines.Line2D at 0x28ffd4eba88>],
 'fliers': [<matplotlib.lines.Line2D at 0x28ffd4eb1c8>],
 'means': []}

In [13]:
cantidad_list = movimientosCopia.select('Cantidad').rdd.flatMap(lambda x: x).collect()

# Plotting a histogram
plt.figure(figsize=(10,6))
plt.hist(cantidad_list, bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Cantidad')
plt.xlabel('Cantidad')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting values <100 
plt.figure(figsize=(10,6))
plt.hist([x for x in cantidad_list if x < 100], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Cantidad (Zoomed In)')
plt.xlabel('Cantidad')
plt.ylabel('Frequency')
plt.grid(True)
plt.show() 

# Plotting values >1 and values <100 
plt.figure(figsize=(10,6))
plt.hist([x for x in cantidad_list if x < 100 and x > 0], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Cantidad (Zoomed In) values >1 and values <100 ')
plt.xlabel('Cantidad')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting values with log scale  
plt.figure(figsize=(10,6))
plt.hist(cantidad_list, bins=50, edgecolor='black', alpha=0.7, log=True)
plt.title('Distribution of Cantidad log scale ')
plt.xlabel('Cantidad')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [14]:
"""
    Considering these graphs, it's unusual to observe negative values. Additionally, 
    there appears to be a distinction between users who purchase only a few items and 
    those who buy a large number of products. It's worth noting that approximately 
    25% of the data falls above -5.0, while the range for 0% to 75% of the values is 
    between -360 and -5. This distribution is less than ideal, especially given the 
    presence of negative values.
"""

"\n    Considering these graphs, it's unusual to observe negative values. Additionally, \n    there appears to be a distinction between users who purchase only a few items and \n    those who buy a large number of products. It's worth noting that approximately \n    25% of the data falls above -5.0, while the range for 0% to 75% of the values is \n    between -360 and -5. This distribution is less than ideal, especially given the \n    presence of negative values.\n"

In [15]:
import missingno as msno 
print(f"Number of total rows: {movimientosCopia.count()}") 
print(f"Number of '' rows: {contar_vacios_str(movimientosCopia, False)[0]}") 
print(f"% of duplicated rows: {movimientosCopia.distinct().count()/movimientosCopia.count()}") 
# Patterns of data completion 
msno.matrix(movimientosCopia.toPandas()) 

Number of total rows: 204292
Number of '' rows: 197182
% of duplicated rows: 0.8500528655062362


<AxesSubplot:>

In [16]:
missing_percentage = movimientosCopia.toPandas().isnull().mean() * 100

for column, percentage in missing_percentage.items():
    print(f"{column}: {percentage:.2f}% missing")

TransaccionProductoID: 0.00% missing
ProductoID: 0.00% missing
TipoTransaccionID: 0.00% missing
ClienteID: 0.00% missing
InvoiceID: 0.00% missing
ProveedorID: 0.00% missing
OrdenDeCompraID: 0.00% missing
FechaTransaccion: 0.00% missing
Cantidad: 0.00% missing


In [17]:
df_summary = movimientosCopia.describe()

# Counting unique values 
unique_counts = {
    'ProductoID': movimientosCopia.select('ProductoID').distinct().count(),
    'TipoTransaccionID': movimientosCopia.select('TipoTransaccionID').distinct().count(),
    'ClienteID': movimientosCopia.select('ClienteID').distinct().count(),
    'InvoiceID': movimientosCopia.select('InvoiceID').distinct().count(),
    'ProveedorID': movimientosCopia.select('ProveedorID').distinct().count(),
    'OrdenDeCompraID': movimientosCopia.select('OrdenDeCompraID').distinct().count(),
    'FechaTransaccion': movimientosCopia.select('FechaTransaccion').distinct().count()
}

# Date range 
date_range = movimientosCopia.agg(
    functions.min('FechaTransaccion').alias('Fecha Minima'),
    functions.max('FechaTransaccion').alias('Fecha Maxima')
)

print("Unique values on each column: ") 
print(unique_counts) 
print("Date range: ") 
date_range.show() 

Unique values on each column: 
{'ProductoID': 227, 'TipoTransaccionID': 3, 'ClienteID': 664, 'InvoiceID': 51831, 'ProveedorID': 4, 'OrdenDeCompraID': 1472, 'FechaTransaccion': 2155}
Date range: 
+--------------------+------------+
|        Fecha Minima|Fecha Maxima|
+--------------------+------------+
|2013-12-31 07:00:...| Sep 30,2015|
+--------------------+------------+



In [18]:
#para valores extremos(subir a calidad)
movimientosCopia.toPandas().hist(column='Cantidad')

array([[<AxesSubplot:title={'center':'Cantidad'}>]], dtype=object)

In [19]:
import pandas as pd 
import seaborn as sns 
# 1. Distribution of `Cantidad`
df_cantidad = movimientosCopia.select("Cantidad").toPandas()
plt.figure(figsize=(10, 6))
sns.histplot(df_cantidad, bins=30, kde=True)
plt.title('Distribution of Cantidad')
plt.show()

# 3. Top Products by Transactions
df_product = movimientosCopia.groupBy('ProductoID').count().sort('count', ascending=False).limit(10).toPandas()
plt.figure(figsize=(10, 6))
sns.barplot(data=df_product, x='ProductoID', y='count')
plt.title('Top 10 Products by Transactions')
plt.show()

# 4. Top Clients by Transactions
df_client = movimientosCopia.groupBy('ClienteID').count().sort('count', ascending=False).limit(10).toPandas()
plt.figure(figsize=(10, 6))
sns.barplot(data=df_client, x='ClienteID', y='count')
plt.title('Top 10 Clients by Transactions')
plt.show()

# 5. Correlation Heatmap
# Since we're just working with a few columns, let's compute correlations on them
numerical_columns = ['TransaccionProductoID', 'ProductoID', 'TipoTransaccionID', 'ClienteID', 'InvoiceID', 'Cantidad']
df_corr = movimientosCopia.select(numerical_columns).toPandas().corr()
plt.figure(figsize=(10, 6))
sns.heatmap(df_corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

In [20]:
"""
    Getting those times when each supplier took part of an transaction 
"""
movimientosCopia.groupBy('FechaTransaccion').pivot('ProveedorID').count().show()

+--------------------+----+----+----+----+
|    FechaTransaccion|    | 1.0| 4.0| 7.0|
+--------------------+----+----+----+----+
|2014-08-28 07:00:...|null|null|   5|   1|
|2014-09-19 07:00:...|null|null|   4|   1|
|         Feb 27,2014|  77|null|null|null|
|2014-11-19 07:00:...|null|null|   6|   3|
|2015-02-18 07:00:...|null|null|   2|   2|
|2015-10-27 07:00:...|null|null|   7|   3|
|2016-01-27 07:00:...|null|null|   4|   4|
|2014-06-26 12:00:...| 214|null|null|null|
|2015-02-12 07:00:...|null|null|   5|   1|
|2014-11-28 12:00:...| 139|null|null|null|
|2015-01-16 07:00:...|null|null|   5|   2|
|         Sep 21,2015| 117|null|   2|   1|
|2014-04-18 12:00:...| 201|null|null|null|
|         Jul 26,2014|  69|null|null|null|
|         May 14,2016|  64|null|null|null|
|         Aug 22,2015|  41|null|null|null|
|         Jan 03,2015|  26|null|null|null|
|         Mar 25,2015| 115|null|null|   1|
|2014-11-20 12:00:...| 101|null|null|null|
|         Oct 14,2014|  84|null|   1|null|
+----------

In [21]:
corr = movimientosCopia.toPandas().corr(method='pearson')
corr.style.background_gradient(cmap='coolwarm')

,TransaccionProductoID,ProductoID,TipoTransaccionID,ClienteID,InvoiceID,Cantidad
TransaccionProductoID,1.000000,0.017905,-0.001819,0.021868,0.859235,0.060616
ProductoID,0.017905,1.000000,0.028463,-0.009326,0.001007,0.028805
TipoTransaccionID,-0.001819,0.028463,1.000000,-0.277915,-0.484808,0.833650
ClienteID,0.021868,-0.009326,-0.277915,1.000000,0.154670,-0.234709
InvoiceID,0.859235,0.001007,-0.484808,0.154670,1.000000,-0.409633
Cantidad,0.060616,0.028805,0.833650,-0.234709,-0.409633,1.000000


### Análisis de calidad de datos

##### Unicidad y validez

In [22]:
print(f"Number of distinct rows: {movimientosCopia.distinct().count()}")
print(f"Number of total rows: {movimientosCopia.count()}") 

""" 
    One could think that every value in TransaccionProductoID is unique 
    and different, but according the upper analysis, there are duplicated 
    values, like 327158. So uniqueness is not fulfilled. 
"""

"""
    In terms of validity, the dates do not entirely have a single format, 
    just as there are empty fields that could be important, such as 
    SupplierID and PurchaseOrderID, which could be important for some 
    cross-checking of information or subsequent analysis.
"""

Number of distinct rows: 173659
Number of total rows: 204292


'\n    In terms of validity, the dates do not entirely have a single format, \n    just as there are empty fields that could be important, such as \n    SupplierID and PurchaseOrderID, which could be important for some \n    cross-checking of information or subsequent analysis.\n'

##### Completitud y validez

In [23]:
print("NULL Values: ")
empty_columns_movimientosCopia = contar_vacios(movimientosCopia) 
print("'' Values: ")
empty_columns_movimientosCopia_str = contar_vacios_str(movimientosCopia)

print("\n Cardinality")
movimientosCopia_c = cardinalidad(movimientosCopia) 
print(f"{movimientosCopia_c}") 

"""
    Checking the columns with "" values, just ProveedorID and OrdenDeCompraID 
    have columns with missing data. It needs to be asked to the client if 
    this is Ok, because it seems important to have the OrdenDeCompraID value 
    and its provider too! 
    
    Regarding cardinality, just TransaccionProductoID is notable, but 
    it seems to be Okay since it's a unique ID. 
"""

NULL Values: 
'' Values: 
Number of "" values for column ProveedorID: 197182
Number of "" values for column OrdenDeCompraID: 197182

 Cardinality
{'TransaccionProductoID': 173659}


'\n    Checking the columns with "" values, just ProveedorID and OrdenDeCompraID \n    have columns with missing data. It needs to be asked to the client if \n    this is Ok, because it seems important to have the OrdenDeCompraID value \n    and its provider too! \n    \n    Regarding cardinality, just TransaccionProductoID is notable, but \n    it seems to be Okay since it\'s a unique ID. \n'

##### Consistencia¶

In [24]:
"""
    As mentioned before on Análisis descriptivo section, it seems that 
    there are negative values in "Cantidad" column. It is found that it 
    is necessary to consult with the client about these values and try 
    to give us a better understanding about this situation so we could 
    check if there are some inconsistencies with this column or maybe 
    another rule that we aren't expecting or taking in count. Also, it 
    could give improve the boksplot graph once it's fixed if needed. 
"""

'\n    As mentioned before on Análisis descriptivo section, it seems that \n    there are negative values in "Cantidad" column. It is found that it \n    is necessary to consult with the client about these values and try \n    to give us a better understanding about this situation so we could \n    check if there are some inconsistencies with this column or maybe \n    another rule that we aren\'t expecting or taking in count. Also, it \n    could give improve the boksplot graph once it\'s fixed if needed. \n'

In [25]:
plt.show()

### Conclusiones

In [ ]:

**Conclusiones generales:**

1. En la tabla WWImportersTransactional.movimientosCopia tenemos información relacionada a las transacciones de importación/distribución junto a algunos detalles como el proveedor, el cliente, el producto, el tipo de transacción, fecha, cantidad y entre otros. 

**Conclusiones de reglas de negocio:**

1. Casi que ninguna de las reglas de negocio se cumple. En el desarrollo de estas se explicó a detalle el porqué. Aunque podría decirse que la primera se puede interpretar como verdadera, ya que no rompe necesariamente el límite dado. Por otro lado, la segunda es mayormente falsa pero tiene un poco de verdadera, ya que sí tiene transacciones en 2013, solo que corresponden al último día de ese año.

**Conclusiones de calidad:** 

1. Se puede observar que el campo de **Cantidad** tiene valores negativos. Podría decirse que esto se debe a que se importa y distribuyen productos, por lo que se podría ver de varias maneras, aunque la más interesante sería que los valores positivos correspondan a las importaciones y los negativos a las de distribución, ya que las positivas tienen un valor alto, correspondiendo a las adquisiciones grandes de productos y las negativas no tienen la misma distribución, correspondiente a las compras por parte de los clientes. Además de esto, el mismo producto se presenta en ambas circunstancias, como el 193, que aparece con el valor más alto de Cantidad y también con negativos.
2. ¿Por qué hay valores vacíos en campos que podrían ser importantes, tales como OrdenDeCompraID y el proveedorID y, más aún que esto sucede al mismo tiempo en ambas columnas. 
3. Como se mencionó anteriormente en la sección 'Análisis Descriptivo', parece que hay valores negativos en la columna 'Cantidad'. Es fundamental consultar con el cliente sobre estos valores para comprender mejor la situación. Esto nos permitirá verificar posibles inconsistencias dentro de esta columna o descubrir reglas inesperadas que quizás no se hayan considerado. Abordar este problema también puede conducir a mejoras en el gráfico de diagrama de caja, si es necesario.
4. En términos de validez, las fechas no tienen un formato consistente. Además, hay campos vacíos que podrían ser cruciales, como SupplierID y PurchaseOrderID. Estos campos podrían desempeñar un papel importante en la verificación cruzada de información o en la realización de análisis posteriores.
5. Se podría pensar que cada valor en TransaccionProductoID es único y diferente, pero según el análisis superior, hay valores duplicados, como 327158, por lo que no se cumple la unicidad.
6. Parece que cada columna tiene datos, pero algunas tienen valores atípicos. Por ejemplo, la columna 'Cantidad' tiene un valor mínimo de -360,0 y un valor máximo de 67368,0. Por supuesto, esto se comprobará más adelante.
7. Hay 197182 filas con valores vacíos. 

**Usando herramientas:**

Seccion general del reporte: 
1. Tenemos 10 variables y 204292 registros, de las cuales todas son numéricas pero la mayoría corresponden a IDs, posiblemente es una tabla relacional pero sin ser normalizada totalmente. 

2. Hay 197182 filas vacías y corresponden a las columnas OrdenDeCompraID y el proveedorID

4. Hay registros duplicados

Sección de variables del reporte: 
    
1. X Considerando estos gráficos, es inusual observar valores negativos. Además, parece haber una distinción entre los usuarios que compran sólo unos pocos artículos y los que compran una gran cantidad de productos. Vale la pena señalar que aproximadamente el 25 % de los datos están por encima de -5,0, mientras que el rango del 0 % al 75 % de los valores está entre -360 y -5. Esta distribución no es ideal, especialmente dada la presencia de valores negativos.

2. Para la variable Cantidad se observó una distribución muy poco usual, teniendo una distribución casi que mayoritaria en sus cuartiles menores al 75%. 

3. La variable TransaccionProductoID tiene alta cardinalidad, pero es OK desde que la transacción debería ser única. Además, no brinda mucho aporte en el modelo. 

Sección de correlación:

1. Parece haber algunas correlaciones algo altas en las últimas columnas pero no deberían ser consideradas porque son columnas numéricas y con identificadores que se suponen únicos o no muy repetidos, por lo que la correlación no es muy importante para el análisis de esta tabla. 

**Conclusiones de consultoria**

Teniendo en cuenta este análisis, no será posible empezar con la solución hasta aclarar los temas con dudas en este paso, más que todo por la duda con el campo de Cantidad.

## Cierre

Completado este tutorial ya sabrá la forma básica de utilizar PySpark. Ya sabe cómo crear DataFrames a partir de datos existentes, cómo seleccionar columnas o filas de este Dataframe, cómo rellenar espacios vacíos, cómo aplicar sus propias funciones a estos datos y los más importante, cómo utilizarlo para realizar una fase de entendimiento de datos.

Las tablas detallesOrdenesCopia y OrdenesCopia estaran disponibles en caso de que desee repetir este tutorial cuantas veces considere necesario

## 7. Información adicional

Si quiere conocer más sobre PySpark la guía más detallada es la documentación oficial, la cual puede encontrar acá: https://spark.apache.org/docs/latest/api/python/index.html <br>
Para ir directamente a la documentación de PySpark SQL, donde está la información sobre los DataFrames: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html <br>

El Capítulo 2 del libro <i>Learn PySpark : Build Python-based Machine Learning and Deep Learning Models, New York: Apress. 2019</i> de Pramod Singh contiene muchos ejemplos útiles, puede encontrarlo en la biblioteca virtual de la universidad.

## 8. Preguntas frecuentes

1. Si en el cálculo de los histogramas o en una de sus UDF obtiene el error: TypeError: unsupported operand type(s) for *: 'decimal.Decimal' and 'float' verifique los tipos de las columnas que está intentando operar y, si alguna es double, conviértala a float.

2. En algunos casos, encontrará también información sobre <i>Pandas_UDF</i>. <i>Pandas_UDF</i> son también User Defined Functions, por lo general los Pandas UDF son más eficientes que los UDF tradicionales, sin embargo, hay un bug con la versión de PySpark y de Java que se está usando, lo que previene la utilización de Pandas_UDF.

3. Si al ejecutar la configuración de la sesión Spark le aparece el error <i>Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*])</i> Reinicie el kernel y vuelva a ejecutar
